In [24]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import imageio
import os

In [25]:
files = sorted(glob.glob('Results/output_*.dat'), key=lambda x: (int(x.split('_')[1]), int(x.split('_')[2].split('.')[0])))
files

['Results/output_10_0.dat',
 'Results/output_10_7.dat',
 'Results/output_10_14.dat',
 'Results/output_10_21.dat',
 'Results/output_10_28.dat',
 'Results/output_10_34.dat',
 'Results/output_10_40.dat',
 'Results/output_10_46.dat',
 'Results/output_10_52.dat',
 'Results/output_10_58.dat',
 'Results/output_20_0.dat',
 'Results/output_20_7.dat',
 'Results/output_20_14.dat',
 'Results/output_20_21.dat',
 'Results/output_20_28.dat',
 'Results/output_20_34.dat',
 'Results/output_20_40.dat',
 'Results/output_20_46.dat',
 'Results/output_20_52.dat',
 'Results/output_20_58.dat',
 'Results/output_30_0.dat',
 'Results/output_30_7.dat',
 'Results/output_30_14.dat',
 'Results/output_30_21.dat',
 'Results/output_30_28.dat',
 'Results/output_30_34.dat',
 'Results/output_30_40.dat',
 'Results/output_30_46.dat',
 'Results/output_30_52.dat',
 'Results/output_30_58.dat',
 'Results/output_40_0.dat',
 'Results/output_40_7.dat',
 'Results/output_40_14.dat',
 'Results/output_40_21.dat',
 'Results/output_40_28

In [26]:
filenames = []

# create folder for frames
if not os.path.exists('frames'):
    os.makedirs('frames')

for idx, file in enumerate(files):
    data = []
    
    # read data from all processes    
    data.append(np.loadtxt(file))
    
    # Combine data
    data = np.vstack(data)
    x = data[:, 0]
    y = data[:, 1]
    z = data[:, 2]
    rho = data[:, 3]
    u = data[:, 4]
    v = data[:, 5]
    w = data[:, 6]
    p = data[:, 7]
    Bx = data[:, 8]
    By = data[:, 9]
    Bz = data[:, 10]
    
    # 3D grid
    x_vals = np.unique(x)
    y_vals = np.unique(y)
    z_vals = np.unique(z)
    NX_total = len(x_vals)
    NY_total = len(y_vals)
    NZ_total = len(z_vals)
    
    # grid
    x_grid = x_vals
    y_grid = y_vals
    X, Y = np.meshgrid(x_grid, y_grid)
    
    # z slice data (middle)
    z_index = NZ_total // 2
    z_value = z_vals[z_index]
    
    # z slice data
    indices = np.where(z == z_value)
    u_slice = u[indices].reshape(NY_total, NX_total)
    v_slice = v[indices].reshape(NY_total, NX_total)
    w_slice = w[indices].reshape(NY_total, NX_total)
    p_slice = p[indices].reshape(NY_total, NX_total)
    
    # create figure
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    im1 = axes[0, 0].imshow(u_slice, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', aspect='auto')
    axes[0, 0].set_title('Velocity U')
    axes[0, 0].set_xlabel('X')
    axes[0, 0].set_ylabel('Y')
    fig.colorbar(im1, ax=axes[0, 0])
    
    im2 = axes[0, 1].imshow(v_slice, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', aspect='auto')
    axes[0, 1].set_title('Velocity V')
    axes[0, 1].set_xlabel('X')
    axes[0, 1].set_ylabel('Y')
    fig.colorbar(im2, ax=axes[0, 1])
    
    im3 = axes[1, 0].imshow(w_slice, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', aspect='auto')
    axes[1, 0].set_title('Velocity W')
    axes[1, 0].set_xlabel('X')
    axes[1, 0].set_ylabel('Y')
    fig.colorbar(im3, ax=axes[1, 0])
    
    im4 = axes[1, 1].imshow(p_slice, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', aspect='auto')
    axes[1, 1].set_title('Pressure P')
    axes[1, 1].set_xlabel('X')
    axes[1, 1].set_ylabel('Y')
    fig.colorbar(im4, ax=axes[1, 1])
    
    fig.suptitle(f'Time Step: {idx}, Z = {z_value}')
    plt.tight_layout()
    
    # save frame image
    frame_filename = f'frames/frame_{idx:04d}.png'
    plt.savefig(frame_filename)
    plt.close(fig)
    
    filenames.append(frame_filename)

# create gif
with imageio.get_writer('mhd_simulation.gif', mode='I', duration=0.5) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)